In [2]:
import os, copy
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import warnings

In [3]:
origin_df = pd.read_csv('/home/inter/chat_MBTI/make_dataset/intent_model_dataset.csv')
df = copy.deepcopy(origin_df)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop(df.loc[df['sentence'].str.contains('\*')].index, axis=0, inplace=True)
df = df.loc[df['sentence'].str.len() <= 100]
df.reset_index(inplace=True, drop=True)


df.loc[(df['sentence'].str.contains('응') & df['sentence'].str.contains('좋아')), 'intent'] = '긍정'
df.loc[(df['sentence'].str.contains('응') & df['sentence'].str.contains('맞아')), 'intent'] = '긍정'

df.loc[(df['sentence'].str.contains('아니') & df['sentence'].str.contains('싫어')), 'intent'] = '부정'


fst_P_key = ['맞어','맞아','마자','마쟈','마저','맞지','응 ','응!','응응 ','웅 ','웅웅 ','웅!','좋아 ','좋다']
fst_N_key = ['아뇨','아니','아닌데','아녀','아니여','아냐','싫어 ','싫어,','싫어!','시러','싫은데','안돼','안되지','안된','별로야']

for i in fst_P_key:
    df.loc[(df['sentence'].str.startswith(i)), 'intent'] = '긍정'
for i in fst_N_key:
    df.loc[(df['sentence'].str.startswith(i)), 'intent'] = '부정'

df

,sentence,intent
0,스페인 마드리드에 여행가면 하고 싶은거 있어?,(지시) 질문하기
1,나는 키키 축구 보고 싶네,(단언) 주장하기
2,축구팀 어디 경기 말이야?,(지시) 질문하기
3,거기 레알 마드리드 홈구장이 있잖아 하하,(단언) 진술하기
4,아 그 베르나베우였나?,(지시) 질문하기
...,...,...
740672,말이라도 하는 게 낫겠지?? 오백 올려 주면 진짜 좋겠다...,(단언) 주장하기
740673,최저시급도 올랐는데 연봉도 당연히 올라야 되는 것 아냐?!,(지시) 질문하기
740674,나는 재밌어서 하루 만에 다 봤는데.,(단언) 주장하기
740675,"헐, 엄청 긴데 피곤했겠다.",(단언) 주장하기


In [4]:
asdf = list(df['intent'].unique())
for i in asdf :
    print(i,df.loc[df['intent']==i,'intent'].count())

(지시) 질문하기 170965
(단언) 주장하기 243605
(단언) 진술하기 213634
긍정 46140
(지시) 충고/제안하기 23482
(표현) 부정감정 표현하기 5047
(언약) 약속하기(제3자와)/(개인적 수준) 10114
부정 11292
(표현) 감사하기 1186
턴토크 사인(관습적 반응) 2577
(지시) 명령/요구하기 2037
(지시) 부탁하기 3118
(표현) 긍정감정 표현하기 4395
(단언) 반박하기 2054
(표현) 인사하기 344
(표현) 사과하기 376
(언약) 거절하기 269
(선언/위임하기) 42


In [5]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [6]:
# token 바구니
N_count = []

# 필요 문장 추출
N_intent = ['(표현) 부정감정 표현하기', '(단언) 반박하기', '(언약) 거절하기', '부정']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intent in N_intent :
    for i in df.loc[(df['intent'] == intent), 'sentence']:
        N_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
N_uni = np.unique(N_count, return_counts=True)

# 고유값 개수 내림차순 정렬
N_cnt = sorted(list(set(N_uni[1])),reverse=True)

# N_many = N_uni[0]에서 검색할 index값 / N_token = 사용 횟수가 많은 token
N_many = []
N_token = []

for i in N_cnt:
    if i > 2700 :
        N_many.append(list(N_uni[1]).index(i))
for j in range(len(N_many)) :
    N_token.append(N_uni[0][N_many[j]])
print(N_many)

tokenizer.decode(N_token)

[1, 13, 2049, 1236, 1049, 1048]


'[CLS]. 아니는이어'

In [7]:
# token 바구니
P_count = []

# 필요 문장 추출
P_intent = ['(표현) 긍정감정 표현하기', '(표현) 감사하기', '긍정']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intent in P_intent :
    for i in df.loc[(df['intent'] == intent), 'sentence']:
        P_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
P_uni = np.unique(P_count, return_counts=True)

# 고유값 개수 내림차순 정렬
P_cnt = sorted(list(set(P_uni[1])),reverse=True)

# N_many = N_uni[0]에서 검색할 index값 / N_token = 사용 횟수가 많은 token
P_many = []
P_token = []

for i in P_cnt:
    if i > 7700 :
        P_many.append(list(P_uni[1]).index(i))
for j in range(len(P_many)) :
    P_token.append(P_uni[0][P_many[j]])
print(P_many)

tokenizer.decode(P_token)

[1, 813, 10990, 13, 1421, 1223, 1288, 1224, 1260, 1017]


'[CLS] 응 맞아.는어도이키 키'

In [8]:
# token 바구니
M_count = []

# 필요 문장 추출
M_intent = ['(단언) 진술하기', '(지시) 충고/제안하기', '(단언) 주장하기', '(지시) 질문하기', '(지시) 부탁하기',
        '(표현) 사과하기', '(지시) 명령/요구하기', '턴토크 사인(관습적 반응)',
       '(언약) 약속하기(제3자와)/(개인적 수준)', '(표현) 인사하기', '(선언/위임하기)']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intenti in range(len(M_intent)):
    if df.loc[(df['intent'] == intent), 'sentence'].count() > 1000:
        for i in (df.loc[(df['intent'] == intent), 'sentence']):
            M_count += tokenizer.encode(i)
    else:
        for i in (df.loc[(df['intent'] == intent), 'sentence']):
            M_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
M_uni = np.unique(M_count, return_counts=True)

# 고유값 개수 내림차순 정렬
M_cnt = sorted(list(set(M_uni[1])),reverse=True)

# M_many = N_uni[0]에서 검색할 index값 / M_token = 사용 횟수가 많은 token
M_many = []
M_token = []

for i in M_cnt:
    if i > 100000 :
        M_many.append(list(M_uni[1]).index(i))
for j in range(len(M_many)) :
    M_token.append(M_uni[0][M_many[j]])
print(M_many)

tokenizer.decode(M_token)

[1, 794, 10743, 13, 1390, 1194, 1259]


'[CLS] 응 맞아.는어도'

In [9]:
# 중복값 제거
print(tokenizer.decode((set(N_token).difference(set(P_token))).difference(set(M_token))))
print(tokenizer.decode((set(P_token).difference(set(N_token))).difference(set(M_token))))

아니
##키 키


In [10]:
N_keyword = ['싫어\.\.', '아니']
P_keyword = ['맞아', '응응', '응\.', '응,']

In [11]:
# 부정키워드 포함 긍정문장 제거
for intent in P_intent:
    for key in N_keyword:
        df.drop(df.loc[(df['intent'] == intent) & df['sentence'].str.contains(key)].index, axis=0, inplace=True)

# 긍정키워드 포함 부정문장 제거
for intent in N_intent:
    for key in P_keyword:
        df.drop(df.loc[(df['intent'] == intent) & df['sentence'].str.contains(key)].index, axis=0, inplace=True)

# 필요한 데이터를 추출하기 위한 DataFrame 생성
temp_df = pd.DataFrame()
temp = pd.DataFrame()
df_ = copy.deepcopy(df)

# 기존 데이터에서 긍정, 부정 문장 제거
for intent in (N_intent+P_intent):
    df_.drop(df_[df_['intent'] == intent].index, axis=0, inplace=True)

# 남은 중립문장 중 필요한만큼 문장 추출
temp_df = pd.concat([temp_df,df_.sample(n=130000)], ignore_index=True)

# 중립문장 중 긍정/부정 키워드 포함한 문장 제거
for key in (N_keyword+P_keyword):
    temp_df.drop(temp_df.loc[temp_df['sentence'].str.contains(key)].index, axis=0, inplace=True)

In [12]:
# 필요한만큼 긍정/부정문장 추출
for intent in (N_intent+P_intent):
    print(intent)
    if df.loc[df['intent']==intent, 'intent'].count() > 100000:
        temp = pd.concat([temp,df[df['intent'] == intent].sample(n=100000)], ignore_index=True)
    # elif df.loc[df['intent']==intent, 'intent'].count() > 10000:
    #     temp = pd.concat([temp,df[df['intent'] == intent].sample(n=10000)], ignore_index=True)
    else :
        temp = pd.concat([temp,df[df['intent'] == intent]], ignore_index=True)

temp = pd.concat([temp,temp_df], ignore_index=True)

print('='*100)
for i in list(temp['intent'].unique()):
  print(i)

print(temp)

temp.loc[(temp['intent'] == '(표현) 부정감정 표현하기')|(temp['intent'] == '(언약) 거절하기')|
         (temp['intent'] == '(단언) 반박하기'), 'intent'] = '부정'
temp.loc[(temp['intent'] == '(표현) 긍정감정 표현하기')|(temp['intent'] == '(표현) 감사하기'), 'intent'] = '긍정'
temp.loc[(temp['intent'] != '부정')&(temp['intent'] != '긍정'), 'intent'] = '중립'

(표현) 부정감정 표현하기
(단언) 반박하기
(언약) 거절하기
부정
(표현) 긍정감정 표현하기
(표현) 감사하기
긍정
(표현) 부정감정 표현하기
(단언) 반박하기
(언약) 거절하기
부정
(표현) 긍정감정 표현하기
(표현) 감사하기
긍정
(지시) 질문하기
(지시) 충고/제안하기
(단언) 진술하기
(단언) 주장하기
(언약) 약속하기(제3자와)/(개인적 수준)
턴토크 사인(관습적 반응)
(지시) 부탁하기
(지시) 명령/요구하기
(표현) 사과하기
(표현) 인사하기
(선언/위임하기)
                          sentence          intent
0                    저런... 정말 힘들겠다  (표현) 부정감정 표현하기
1                        습한 느낌이 싫어  (표현) 부정감정 표현하기
2                  대단한데? 난 무서워서 못해  (표현) 부정감정 표현하기
3                    그런거같아 너무 머리아파  (표현) 부정감정 표현하기
4             진료 받는 것도 무섭고 돈도 무서워~  (표현) 부정감정 표현하기
...                            ...             ...
195415        요즘 아르바이트 하니 예전 생각 난다       (단언) 진술하기
195416               그래 처음에는 그랬겠구먼       (단언) 진술하기
195417   팔 흔들고 다니는건 무리하게 되는게 아닌걸까?       (지시) 질문하기
195418        찌개 맛 때문에 별 차이 없지 않아?       (지시) 질문하기
195419  아무리 신차라도 판매도 못하는걸 전시할라나...       (단언) 주장하기

[195420 rows x 2 columns]


In [13]:
fin_df = copy.deepcopy(temp)

In [14]:
temp = list(fin_df['intent'].unique())
for i in temp :
    print(i,fin_df.loc[fin_df['intent']==i,'intent'].count())

부정 18581
긍정 50929
중립 125910


In [15]:
# 클래수 개수
lable_num = len(fin_df['intent'].unique())
lable_num

3

In [16]:
label_dict = {0: '긍정', 1: '부정', 2: '중립'}

for idx, intent_lab in label_dict.items() :
    fin_df.loc[fin_df['intent'] == intent_lab, 'intent'] = idx

print(label_dict)
fin_df = fin_df.sample(frac=1).reset_index(drop=True)
fin_df

{0: '긍정', 1: '부정', 2: '중립'}


,sentence,intent
0,웃겨 증말,2
1,나 그거 1편 밖에 안 봤어,2
2,어때 반대해? 찬성해,2
3,맞아 키키 거긴 사람들이 다 평점 좋아,0
4,난또보고싶은데,2
...,...,...
195415,말년 휴가는 꼭 나와야지 당연히 ㅠㅠ,2
195416,혹시 요리중에 어떤 걸 잘하세요,2
195417,야근도 시켜 거긴?,2
195418,쇼핑몰 쿠폰 잘 주는 데 알아?,2


In [17]:
train_data = pd.DataFrame()

len_ = fin_df['intent'].count()

train_data = pd.concat([train_data,fin_df.loc[:(len_*8)/10]],ignore_index=True)
print(train_data)

                     sentence intent
0                       웃겨 증말      2
1             나 그거 1편 밖에 안 봤어      2
2                 어때 반대해? 찬성해      2
3       맞아 키키 거긴 사람들이 다 평점 좋아      0
4                     난또보고싶은데      2
...                       ...    ...
156332            아 그거 마약 아녀?      2
156333             벚꽃 구경도 못했다      2
156334         라고 소스인데 흰 버전이야      2
156335       웅 올해 받으면 4년째라던데?      0
156336     일렉기타는 너무 시끄럽지 않을까?      2

[156337 rows x 2 columns]


In [18]:
test_data = pd.DataFrame()

len_ = fin_df['intent'].count()

test_data = pd.concat([test_data,fin_df.loc[(len_*8)/10:]],ignore_index=True)
print(test_data)

                     sentence intent
0          일렉기타는 너무 시끄럽지 않을까?      2
1      맞아 나이키 공식 홈페이지에서 배송해준대      0
2             왜 키키 물을 소독을 해서?      2
3           응 엄마가 등산을 좋아하시더라고      0
4      가수는 당연히 노래 잘 하는 게 중요하지      0
...                       ...    ...
39079    말년 휴가는 꼭 나와야지 당연히 ㅠㅠ      2
39080       혹시 요리중에 어떤 걸 잘하세요      2
39081              야근도 시켜 거긴?      2
39082       쇼핑몰 쿠폰 잘 주는 데 알아?      2
39083  그렇지 경제학과 미적분 못해서 난리였잖아      2

[39084 rows x 2 columns]


In [19]:
max_seq_len = max(df['sentence'].apply(lambda x: len(str(x))))

sent1 = train_data['sentence'].iloc[0]
print(max_seq_len)
print('문 장1 :',sent1)

100
문 장1 : 웃겨 증말


In [32]:
# 정수인코딩
encoded_result = tokenizer(sent1, max_length=max_seq_len, padding='max_length', truncation=True)['input_ids']
print(encoded_result)
print('길 이 :', len(encoded_result))

[2, 18281, 1582, 2231, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길 이 : 100


In [21]:
# 세그멘트인코딩
print([0]*max_seq_len)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [22]:
# 어텐션마스크인코딩
valid_num = len(tokenizer.encode(sent1))
print(valid_num * [1] + (max_seq_len - valid_num) * [0])

[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [33]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드임베딩을 위한 문장의 정수인코딩
        input_id = tokenizer(example, max_length=max_seq_len, padding='max_length', truncation=True)['input_ids']

        # attention_mask는 실제단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 세그먼트인코딩
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length{} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [34]:
train_X, train_y = convert_examples_to_features(train_data['sentence'], train_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 156337/156337 [00:19<00:00, 8109.72it/s]


In [35]:
test_X, test_y = convert_examples_to_features(test_data['sentence'], test_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/39084 [00:00<?, ?it/s]

100%|██████████| 39084/39084 [00:04<00:00, 8479.82it/s]


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=lable_num, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

I0000 00:00:1763233912.622286   11643 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1920 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:01:00.0, compute capability: 8.6
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
model.fit(train_X, train_y, epochs=4, batch_size=128, validation_split=0.2)

Epoch 1/4
1144/1144 [==============================] - 365s 280ms/step - loss: 0.1897 - accuracy: 0.9366 - val_loss: 0.1679 - val_accuracy: 0.9431
Epoch 2/4
1144/1144 [==============================] - 308s 269ms/step - loss: 0.1533 - accuracy: 0.9454 - val_loss: 0.1702 - val_accuracy: 0.9446
Epoch 3/4
1144/1144 [==============================] - 308s 269ms/step - loss: 0.1228 - accuracy: 0.9527 - val_loss: 0.1818 - val_accuracy: 0.9390
Epoch 4/4
1144/1144 [==============================] - 308s 269ms/step - loss: 0.0820 - accuracy: 0.9679 - val_loss: 0.2252 - val_accuracy: 0.9286


In [ ]:
results = model.evaluate(test_X, test_y, batch_size=64)
print("test loss, test acc: ", results)

715/715 [==============================] - 38s 53ms/step - loss: 0.2238 - accuracy: 0.9285
test loss, test acc:  [0.22375456988811493, 0.9285027384757996]


In [ ]:
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]

    score = np.argmax(model.predict(encoded_input)[0])

    print('sentence :',new_sentence)
    print(label_dict[score])

In [ ]:
MODEL_SAVE_PATH = os.path.join("/content/Drive/MyDrive/NP_classifier/NP_classifier")
print(MODEL_SAVE_PATH)

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")

else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

/content/Drive/MyDrive/NP_classifier/NP_classifier
/content/Drive/MyDrive/NP_classifier/NP_classifier -- Folder create complete 



('/content/Drive/MyDrive/NP_classifier/NP_classifier/tokenizer_config.json',
 '/content/Drive/MyDrive/NP_classifier/NP_classifier/special_tokens_map.json',
 '/content/Drive/MyDrive/NP_classifier/NP_classifier/vocab.txt',
 '/content/Drive/MyDrive/NP_classifier/NP_classifier/added_tokens.json')

In [ ]:
new_sentence = input('sentence > ')
sentiment_predict(new_sentence)

sentence > 인공지능 재미있네!
1/1 [==============================] - 0s 43ms/step
sentence : 인공지능 재미있네!
긍정
